In [1]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 3
max_degree = 10

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np
from drnet_em import generate_all_binary_vectors

adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0])  # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

3 10


100%|██████████| 320/320 [00:00<00:00, 386.96it/s]


(0.48054906195187586,
 0.5821875000000001,
 0.331125,
 0.32397469144445507,
 0.3563344253845968)

In [3]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:00<00:00, 582.67it/s]


{'average': 0.3122125,
 'direct_effect': 0.24641249999999998,
 'spillover_effect': -0.0899625,
 'psi_1_gamma': 0.387825,
 'psi_0_gamma': 0.1414125,
 'psi_zero': 0.231375}

In [5]:
from drnet_em import *

idx = 2
doubly_robust_em(A_chain[idx], L_chain[idx], Y_chain[idx], adj, return_raw=False)

{'average': 0.29657295281740487,
 'direct_effect': 0.33752569986923564,
 'spillover_effect': 0.06030496290893989,
 'psi_1_gamma': 0.3978306627781755,
 'psi_0_gamma': 0.06030496290893989,
 'psi_zero': 0.0}

In [7]:
from run_pll import *

run_dr_em(A_chain, L_chain, Y_chain, adj, idx, mispec=None)

array([0.29657295, 0.3375257 , 0.06030496, 0.39783066, 0.06030496,
       0.        ])

In [8]:
from tqdm import tqdm

res_list_dr = []
for i in tqdm(range(100)):
    res = run_dr_em(A_chain, L_chain, Y_chain, adj, i, mispec=None)
    res_list_dr.append(res)
res_list_dr = np.array(res_list_dr)

res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")


100%|██████████| 100/100 [07:41<00:00,  4.62s/it]

average: 0.30629 ± 0.03836
direct_effect: 0.25662 ± 0.08329
spillover_effect: 0.07097 ± 0.21407
psi_1_gamma: 0.38328 ± 0.04212
psi_0_gamma: 0.12666 ± 0.07498
psi_zero: 0.05569 ± 0.20625


In [ ]:
from run_pll import *

run_dr(A_chain, L_chain, Y_chain, adj, idx, mispec=None)

array([0.30723371, 0.25791312, 0.01892909, 0.3911775 , 0.13326438,
       0.1143353 ])

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr at 0x1047a04c0> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.44385 ± 0.20028
direct_effect: 0.31246 ± 0.32717
spillover_effect: 0.03181 ± 0.35881
psi_1_gamma: 0.53812 ± 0.26768
psi_0_gamma: 0.22566 ± 0.19122
psi_zero: 0.19385 ± 0.29674
